In [1]:
import os
import numpy as np 
import pandas as pd 

In [2]:
!wget -P ./../../../dataset/ -nc https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/airquality_no2.csv

--2022-12-07 13:11:42--  https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/airquality_no2.csv
Resolving cleanlab-public.s3.amazonaws.com (cleanlab-public.s3.amazonaws.com)... 52.216.29.196, 54.231.133.217, 52.216.114.99, ...
Connecting to cleanlab-public.s3.amazonaws.com (cleanlab-public.s3.amazonaws.com)|52.216.29.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 645534 (630K) [text/csv]
Saving to: './../../../dataset/airquality_no2.csv'

airquality_no2.csv  100%[===================>] 630.40K   649KB/s    in 1.0s    

2022-12-07 13:11:44 (649 KB/s) - './../../../dataset/airquality_no2.csv' saved [645534/645534]



In [3]:
path = "./../../../dataset/airquality_no2.csv"
dataset_name = os.path.splitext(os.path.basename(path))[0]
data = pd.read_csv(path, index_col=0)
print("shape of data: ", data.shape)
data.head() 

shape of data:  (7393, 9)


,time_slot,day_of_week,PT08.S2(NMHC),PT08.S3(NOx),PT08.S5(O3),PT08.S1(CO),true_label,given_label,true_error
0,evening,wednesday,1046.0,1056.0,1268.0,1360.0,0.017948,0.678878,0
1,evening,wednesday,955.0,1174.0,972.0,1292.0,-0.422958,0.302648,0
2,evening,wednesday,939.0,1140.0,1074.0,1402.0,0.038944,0.291333,0
3,evening,wednesday,948.0,1092.0,1203.0,1376.0,0.206908,0.373368,0
4,night,wednesday,836.0,1205.0,1110.0,1272.0,0.080935,0.107462,0


In [4]:
training_data = data.drop(['true_label', 'true_error'], axis=1)
training_data.head(2)

,time_slot,day_of_week,PT08.S2(NMHC),PT08.S3(NOx),PT08.S5(O3),PT08.S1(CO),given_label
0,evening,wednesday,1046.0,1056.0,1268.0,1360.0,0.678878
1,evening,wednesday,955.0,1174.0,972.0,1292.0,0.302648


In [5]:
y = training_data['given_label']
X = training_data.drop(['given_label'], axis=1)

###### Add your training code here ###############
# model = 
# model_fit = model.fit(X,y)
# predictions = model.predict(X)

##### saving the predictions ####################
# savepath = "./../predictions/model_predictions.npy"
# np.save(savepath, np.array(predictions))

## Section section is relevant only if using AutoGluon

Uncomment the code in following cells and run to generate output and save to predictions folder. 

### set up arguments 

In [6]:
from autogluon.tabular import TabularPredictor

data_without_label = training_data.drop(['given_label'], axis=1)
savepath = "./../trained_models/"
if not os.path.isdir(savepath):
    os.mkdir(savepath)

predictorArgs = {
                    "path"             : savepath, 
                    "label"            : 'given_label',
                    "eval_metric"      : 'r2',
                    "problem_type"     : 'regression'
                }

hyperparameters = { 
                    'GBM'   : {}, 
                    'FASTAI': {}, 
                    'RF'    : {'criterion': 'squared_error', 
                                'ag_args': {'name_suffix': 'MSE', 
                                'problem_types': ['regression', 'quantile']}}
                    }

### Normal Model fitting

In [7]:
predictor = TabularPredictor(**predictorArgs)
predictor.fit(training_data, hyperparameters=hyperparameters)
leaderboard = predictor.leaderboard(training_data, silent=True)
leaderboard

Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    7393
Train Data Columns: 6
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1438.9 MB
	Train Data (Original)  Memory Usage: 1.18 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtype

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE,0.953748,0.804547,0.208881,0.037635,1.120197,0.208881,0.037635,1.120197,1,True,2
1,WeightedEnsemble_L2,0.886030,0.831635,0.282430,0.046528,5.959409,0.022641,0.000180,0.027280,2,True,4
2,LightGBM,0.849392,0.791131,0.024958,0.004174,2.449436,0.024958,0.004174,2.449436,1,True,1
3,NeuralNetFastAI,0.849378,0.828854,0.050908,0.008713,4.811932,0.050908,0.008713,4.811932,1,True,3


### Cross Validation fitting

In [8]:
predictor_cv = TabularPredictor(**predictorArgs) 
predictor_cv.fit(training_data, presets="best_quality", num_stack_levels= 0, hyperparameters=hyperparameters)
leaderboard_cv = predictor_cv.leaderboard(training_data, silent=True)
leaderboard_cv

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    7393
Train Data Columns: 6
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1381.17 MB
	Train Data (Original)  Memory Usage: 1.18 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerat

[1000]	valid_set's l2: 0.209502	valid_set's r2: 0.708557


	0.7921	 = Validation score   (r2)
	18.6s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	0.7936	 = Validation score   (r2)
	1.15s	 = Training   runtime
	0.19s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set numb

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE_BAG_L1,0.972010,0.793608,0.233703,0.194284,1.153526,0.233703,0.194284,1.153526,1,True,2
1,LightGBM_BAG_L1,0.905915,0.792106,0.579165,0.085094,18.602896,0.579165,0.085094,18.602896,1,True,1
2,WeightedEnsemble_L2,0.884075,0.829168,0.649634,0.258923,30.447721,0.002015,0.000231,0.055435,2,True,4
3,NeuralNetFastAI_BAG_L1,0.852918,0.827387,0.413916,0.064408,29.238759,0.413916,0.064408,29.238759,1,True,3


### predicting and saving arrays 

In [ ]:
models = {  'normal'            : list(leaderboard['model']), 
            'crossValidation'   : list(np.append(leaderboard_cv['model'], ['oof']))}

predictors = { 'normal'          : predictor, 
               'crossValidation' : predictor_cv}

# check if path is available, else create it. 
pred_path = "./../predictions/"
if not os.path.isdir(pred_path):
    os.mkdir(pred_path)

for model_type in models.keys():
    if model_type == "normal": 
        for model in models[model_type]:
            savepath = os.path.join(pred_path, model+".npy")
            predictions = predictors[model_type].predict(data_without_label, model=model)
            np.save(savepath, np.array(predictions))
    elif model_type == "crossValidation":
        for model in models[model_type]:
            if model == "oof":
                predictions = predictors[model_type].get_oof_pred()
                savepath = os.path.join(pred_path, model+".npy")
            elif model in models['normal']:
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+"_CV.npy")
            else: 
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+".npy")

            np.save(savepath, np.array(predictions))